In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import scipy as sp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/life-expectancy-who/Life Expectancy Data.csv')
data.head()

In [ ]:
data.columns

In [ ]:
data.dataframeName = 'Life Expectancy Data.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
plt.figure(figsize=(10,4))
sns.set_theme(style="darkgrid")
sns.countplot(data=data,x='Year')

In [ ]:
plt.figure(figsize=(10,4))
sns.countplot(data=data,x='Status')

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
a = list(data.columns)
b = []
for i in a:
    c = data[i].isnull().sum()
    b.append(c)
null_df = pd.DataFrame({'Feature name':a,'No. of Null Values':b})
null_df.sort_values(by='No. of Null Values',ascending=False)

In [ ]:
# This piece of code will fill the null values of the selected feature with its mean.
# This one is for filling Nans in 'Life expectancy' column.

data['Life expectancy '] = data['Life expectancy '].fillna(value=data['Life expectancy '].mean())

# Lets also do the same for Adult Mortality.

data['Adult Mortality'] = data['Adult Mortality'].fillna(value=data['Adult Mortality'].mean())

In [ ]:
lifeExpectancy = data.pop('Life expectancy ')
lifeExpectancy.head()

In [ ]:
corr_data = data.corr()
corr_data['Alcohol'].sort_values(ascending=False)[:2]

In [ ]:
sns.heatmap(corr_data)

In [ ]:
sns.scatterplot(x=data['Schooling'],y=data['Alcohol'])

In [ ]:
# These values are mean values of the selected interval of other feature.
rangeval=[]
rangeval.append(data[data['Schooling']<=2.5]['Alcohol'].mean())
rangeval.append(data[(2.5<data['Schooling'] ) & (data['Schooling']<=5.0)]['Alcohol'].mean())
rangeval.append(data[(5.0<data['Schooling'] ) & (data['Schooling']<=7.5)]['Alcohol'].mean())
rangeval.append(data[(7.5<data['Schooling'] ) & (data['Schooling']<=10.0)]['Alcohol'].mean())
rangeval.append(data[(10.0<data['Schooling'] ) & (data['Schooling']<=15.0)]['Alcohol'].mean())
rangeval.append(data[data['Schooling']>15.0]['Alcohol'].mean())
print("sc<=25: ",rangeval[0])
print("2.5<sc<=5.0: ",rangeval[1])
print("5.0<sc<=7.5: ",rangeval[2])
print("7.5<sc<=10.0: ",rangeval[3])
print("10.0<sc<=15: ",rangeval[4])
print("sc>15: ",rangeval[5])

def impute_Alcohol(cols):
    al=cols[0]
    sc=cols[1]
    if pd.isnull(al):
        if sc<=2.5:
            return  rangeval[0]
        elif 2.5<sc<=5.0:
            return rangeval[1]
        elif 5.0<sc<=7.5:
            return rangeval[2]
        elif 7.5<sc<=10.0:
            return rangeval[3]
        elif 10.0<sc<=15:
            return rangeval[4]
        elif sc>15:
            return rangeval[5]
    else:
        return al
    
data['Alcohol']=data[['Alcohol','Schooling']].apply(impute_Alcohol,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
# the leftover nans are due to them being nan's in the schooling column
# populate it with mean values
data['Alcohol'] = data['Alcohol'].fillna(value=data['Alcohol'].mean())
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
sns.distplot(data['Alcohol']);

There is a huge positive skew for this distribution. Lets look at its Kurtosis value.

In [ ]:
sp.stats.skew(data['Alcohol'],axis=0)

Repeat the nan replacement algorithm same way

In [ ]:
corr_data['Diphtheria '].sort_values(ascending=False)[:3]

In [ ]:
corr_data['Polio'].sort_values(ascending=False)[:3]

Since there are nan values at the same places for Diptheria and Polio, and total nan values are only 20 we fill each with their mean values

In [ ]:
data['Diphtheria '] = data['Diphtheria '].fillna(value=data['Diphtheria '].mean())

# Lets also do the same for Polio.

data['Polio'] = data['Polio'].fillna(value=data['Polio'].mean())

In [ ]:
corr_data['Hepatitis B'].sort_values(ascending=False)[:3]

In [ ]:
sns.scatterplot(x=data['Diphtheria '],y=data['Hepatitis B']);

In [ ]:
# These values are mean values of the selected interval of other feature.
rangeval=[]
rangeval.append(data[data['Diphtheria ']<=15]['Hepatitis B'].mean())
rangeval.append(data[(15<data['Diphtheria '] ) & (data['Diphtheria ']<=30)]['Hepatitis B'].mean())
rangeval.append(data[(30<data['Diphtheria '] ) & (data['Diphtheria ']<=45)]['Hepatitis B'].mean())
rangeval.append(data[(45<data['Diphtheria '] ) & (data['Diphtheria ']<=60)]['Hepatitis B'].mean())
rangeval.append(data[(60<data['Diphtheria '] ) & (data['Diphtheria ']<=80)]['Hepatitis B'].mean())
rangeval.append(data[data['Diphtheria ']>80]['Hepatitis B'].mean())
print("sc<=25: "   ,rangeval[0])
print("15<sc<=30: ",rangeval[1])
print("30<sc<=45: ",rangeval[2])
print("45<sc<=60: ",rangeval[3])
print("60<sc<=80: ",rangeval[4])
print("sc>80: "    ,rangeval[5])
def impute_HepatatisB(cols):
    hep=cols[0]
    dip=cols[1]
    if pd.isnull(hep):
        if dip<=15:
            return rangeval[0]
        elif 15<dip<=30:
            return rangeval[1]
        elif 30<dip<=45:
            return rangeval[2]
        elif 45<dip<=60:
            return rangeval[3]
        elif 60<dip<=80:
            return rangeval[4]
        elif dip>80:
            return rangeval[5]
    else:
        return hep
    
data['Hepatitis B']=data[['Hepatitis B','Diphtheria ']].apply(impute_HepatatisB,axis=1)


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
corr_data['Population'].sort_values(ascending=False)[:3]

In [ ]:
sns.scatterplot(x=data['infant deaths'],y=data['Population']);

In [ ]:
rangeval=[]
rangeval.append(data[data['infant deaths']<=100]['Population'].mean())
rangeval.append(data[(100<data['infant deaths'] ) & (data['infant deaths']<=250)]['Population'].mean())
rangeval.append(data[(250<data['infant deaths'] ) & (data['infant deaths']<=350)]['Population'].mean())
rangeval.append(data[(350<data['infant deaths'] ) & (data['infant deaths']<=900)]['Population'].mean())
rangeval.append(data[(900<data['infant deaths'] ) & (data['infant deaths']<=1100)]['Population'].mean())
rangeval.append(data[(1100<data['infant deaths'] ) & (data['infant deaths']<=1250)]['Population'].mean())
rangeval.append(data[(1250<data['infant deaths'] ) & (data['infant deaths']<=1500)]['Population'].mean())
rangeval.append(data[(1500<data['infant deaths'] ) & (data['infant deaths']<=1750)]['Population'].mean())
rangeval.append(data[data['infant deaths']>1750]['Population'].mean())
print("sc<=25: "       ,rangeval[0])
print("100<sc<=250: "  ,rangeval[1])
print("250<sc<=350: "  ,rangeval[2])
print("350<sc<=900: "  ,rangeval[3])
print("900<sc<=1100: " ,rangeval[4])
print("1100<sc<=1250: ",rangeval[5])
print("1250<sc<=1500 " ,rangeval[6])
print("1500<sc<=1750: ",rangeval[7])
print("sc>1750: "      ,rangeval[8])
def impute_population(c):
    p=c[0]
    i=c[1]
    if pd.isnull(p):
        if i<=100:
            return  rangeval[0]
        elif 100<i<=250:
            return  rangeval[1]
        elif 250<i<=350:
            return  rangeval[2]
        elif 350<i<=900:
            return  rangeval[3]
        elif 900<i<=1100:
            return  rangeval[4]
        elif 1100<i<=1250:
            return  rangeval[5]
        elif 1250<i<=1500:
            return  rangeval[6]
        elif 1500<i<=1750:
            return  rangeval[7]
        elif i>1750:
            return  rangeval[8]
    else:
        return p
    
data['Population']=data[['Population','infant deaths']].apply(impute_population,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
corr_data['Total expenditure'].sort_values(ascending=False)[:3]

In [ ]:
sns.scatterplot(x=data['Alcohol'],y=data['Total expenditure']);

In [ ]:
rangeval=[]
rangeval.append(data[data['Alcohol']<=2.5]['Total expenditure'].mean())
rangeval.append(data[(2.5<data['Alcohol'] ) & (data['Alcohol']<=5.0)]['Total expenditure'].mean())
rangeval.append(data[(5.0<data['Alcohol'] ) & (data['Alcohol']<=10.0)]['Total expenditure'].mean())
rangeval.append(data[(10.0<data['Alcohol'] ) & (data['Alcohol']<=12.5)]['Total expenditure'].mean())
rangeval.append(data[data['Alcohol']>12.5]['Total expenditure'].mean())
print("sc<=25: "       ,rangeval[0])
print("2.5<sc<=5.0: "  ,rangeval[1])
print("5.0<sc<=10.0: " ,rangeval[2])
print("10.0<sc<=12.5: ",rangeval[3])
print("sc>12.5: "      ,rangeval[4])

def impute_Total_exp(c):
    t=c[0]
    a=c[1]
    if pd.isnull(t):
        if a<=2.5:
            return rangeval[0]
        elif 2.5<a<=5.0:
            return rangeval[1]
        elif 5.0<a<=10.0:
            return rangeval[2]
        elif 10.0<a<=12.5:
            return rangeval[3]
        elif a>12.5:
            return rangeval[4]
    else:
        return t
    
data['Total expenditure']=data[['Total expenditure','Alcohol']].apply(impute_Total_exp,axis=1) 

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
corr_data['GDP'].sort_values(ascending=False)[:3]

In [ ]:
sns.scatterplot(x=data['percentage expenditure'],y=data['GDP']);

In [ ]:
rangeval=[]
rangeval.append(data[data['percentage expenditure']<=1250]['GDP'].mean())
rangeval.append(data[(1250<data['percentage expenditure'] ) & (data['percentage expenditure']<=2500)]['GDP'].mean())
rangeval.append(data[(2500<data['percentage expenditure'] ) & (data['percentage expenditure']<=3750)]['GDP'].mean())
rangeval.append(data[(3750<data['percentage expenditure'] ) & (data['percentage expenditure']<=7500)]['GDP'].mean())
rangeval.append(data[(7500<data['percentage expenditure'] ) & (data['percentage expenditure']<=8750)]['GDP'].mean())
rangeval.append(data[(8750<data['percentage expenditure'] ) & (data['percentage expenditure']<=10000)]['GDP'].mean())
rangeval.append(data[(10000<data['percentage expenditure'] ) & (data['percentage expenditure']<=11250)]['GDP'].mean())
rangeval.append(data[(11250<data['percentage expenditure'] ) & (data['percentage expenditure']<=12500)]['GDP'].mean())
rangeval.append(data[(12500<data['percentage expenditure'] ) & (data['percentage expenditure']<=15000)]['GDP'].mean())
rangeval.append(data[(15000<data['percentage expenditure'] ) & (data['percentage expenditure']<=17500)]['GDP'].mean())
rangeval.append(data[data['percentage expenditure']>17500]['GDP'].mean())
print("sc<=1250: "      ,rangeval[0])
print("1250 <sc<=2500: ",rangeval[1])
print("2500 <sc<=3750: ",rangeval[2])
print("3750 <sc<=7500: ",rangeval[3])
print("7500 <sc<=8750: ",rangeval[4])
print("8750 <sc<=10000:",rangeval[5])
print("10000<sc<=11250:",rangeval[6])
print("11250<sc<=12500:",rangeval[7])
print("12500<sc<=15000:",rangeval[8])
print("15000<sc<=17500:",rangeval[9])
print("sc>17500:",rangeval[10])

def impute_GDP(c):
    g=c[0]
    p=c[1]
    if pd.isnull(g):
        if p<=1250:
            return rangeval[0]
        elif 1250<p<=2500:
            return rangeval[1]
        elif 2500<p<=3750:
            return rangeval[2]
        elif 3750<p<=7500:
            return rangeval[3]
        elif 7500<p<=8750:
            return rangeval[4]
        elif 8750<p<=10000:
            return rangeval[5]
        elif 10000<p<=11250:
            return rangeval[6]
        elif 11250<p<=12500:
            return rangeval[7]
        elif 12500<p<=15000:
            return rangeval[8]
        elif 15000<p<=17500:
            return rangeval[9]
        elif p>17500:
            return rangeval[10]
    else:
        return g
    
data['GDP']=data[['GDP','percentage expenditure']].apply(impute_GDP,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
corr_data['Schooling'].sort_values(ascending=False)[:3]

In [ ]:
corr_data[' thinness  1-19 years'].sort_values(ascending=False)[:3]

In [ ]:
corr_data['Income composition of resources'].sort_values(ascending=False)[:3]

In [ ]:
corr_data[' BMI '].sort_values(ascending=False)[:4]

In [ ]:
a = list(data.columns)
b = []
for i in a:
    c = data[i].isnull().sum()
    b.append(c)
null_df = pd.DataFrame({'Feature name':a,'No. of Null Values':b})
null_df.sort_values(by='No. of Null Values',ascending=False)

* Use schooling to impute BMI
* Use BMI to impute income
* Use income to impute schooling

In [ ]:
sns.scatterplot(x=data['Schooling'],y=data[' BMI ']);

In [ ]:
rangeval=[]
rangeval.append(data[data['Schooling']<=2.5][' BMI '].mean())
rangeval.append(data[(2.5<data['Schooling'] ) & (data['Schooling']<=5.0)][' BMI '].mean())
rangeval.append(data[(5.0<data['Schooling'] ) & (data['Schooling']<=7.5)][' BMI '].mean())
rangeval.append(data[(7.5<data['Schooling'] ) & (data['Schooling']<=10.0)][' BMI '].mean())
rangeval.append(data[(10<data['Schooling'] ) & (data['Schooling']<=15.0)][' BMI '].mean())
rangeval.append(data[data['Schooling']>15.0][' BMI '].mean())
print("sc<=25: ",rangeval[0])
print("2.5<sc<=5.0: ",rangeval[1])
print("5.0<sc<=7.5: ",rangeval[2])
print("7.5<sc<=10.0: ",rangeval[3])
print("10.0<sc<=15: ",rangeval[4])
print("sc>15: ",rangeval[5])

def impute_BMI(cols):
    al=cols[0]
    bmi=cols[1]
    if pd.isnull(al):
        if bmi<=2.5:
            return  rangeval[0]
        elif 2.5<bmi<=5.0:
            return rangeval[1]
        elif 5.0<bmi<=7.5:
            return rangeval[2]
        elif 7.5<bmi<=10.0:
            return rangeval[3]
        elif 10.0<bmi<=15:
            return rangeval[4]
        elif bmi>15:
            return rangeval[5]
    else:
        return al
    
data[' BMI ']=data[[' BMI ','Schooling']].apply(impute_BMI,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
sns.scatterplot(x=data[' BMI '],y=data['Income composition of resources']);

In [ ]:
rangeval=[]
rangeval.append(data[data[' BMI ']<=15]['Income composition of resources'].mean())
rangeval.append(data[(15<data[' BMI '] ) & (data[' BMI ']<=30)]['Income composition of resources'].mean())
rangeval.append(data[(30<data[' BMI '] ) & (data[' BMI ']<=45)]['Income composition of resources'].mean())
rangeval.append(data[(45<data[' BMI '] ) & (data[' BMI ']<=60)]['Income composition of resources'].mean())
rangeval.append(data[(60<data[' BMI '] ) & (data[' BMI ']<=80)]['Income composition of resources'].mean())
rangeval.append(data[data[' BMI ']>80]['Income composition of resources'].mean())
print("sc<=25: "   ,rangeval[0])
print("15<sc<=30: ",rangeval[1])
print("30<sc<=45: ",rangeval[2])
print("45<sc<=60: ",rangeval[3])
print("60<sc<=15: ",rangeval[4])
print("sc>15: "    ,rangeval[5])
def impute_ICR(cols):
    hep=cols[0]
    dip=cols[1]
    if pd.isnull(hep):
        if dip<=15:
            return rangeval[0]
        elif 15<dip<=30:
            return rangeval[1]
        elif 30<dip<=45:
            return rangeval[2]
        elif 45<dip<=60:
            return rangeval[3]
        elif 60<dip<=80:
            return rangeval[4]
        elif dip>80:
            return rangeval[5]
    else:
        return hep
    
data['Income composition of resources']=data[['Income composition of resources',' BMI ']].apply(impute_ICR,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
sns.scatterplot(x=data['Income composition of resources'],y=data['Schooling']);

In [ ]:
rangeval=[]
rangeval.append(data[data['Income composition of resources']<=15]['Schooling'].mean())
rangeval.append(data[(15<data['Income composition of resources'] ) & (data['Income composition of resources']<=30)]['Schooling'].mean())
rangeval.append(data[(30<data['Income composition of resources'] ) & (data['Income composition of resources']<=45)]['Schooling'].mean())
rangeval.append(data[(45<data['Income composition of resources'] ) & (data['Income composition of resources']<=60)]['Schooling'].mean())
rangeval.append(data[(60<data['Income composition of resources'] ) & (data['Income composition of resources']<=80)]['Schooling'].mean())
rangeval.append(data[data['Income composition of resources']>80]['Schooling'].mean())
print("sc<=25: "   ,rangeval[0])
print("15<sc<=30: ",rangeval[1])
print("30<sc<=45: ",rangeval[2])
print("45<sc<=60: ",rangeval[3])
print("60<sc<=80: ",rangeval[4])
print("sc>80: "    ,rangeval[5])
def impute_ICR(cols):
    hep=cols[0]
    dip=cols[1]
    if pd.isnull(hep):
        if dip<=15:
            return rangeval[0]
        elif 15<dip<=30:
            return rangeval[1]
        elif 30<dip<=45:
            return rangeval[2]
        elif 45<dip<=60:
            return rangeval[3]
        elif 60<dip<=80:
            return rangeval[4]
        elif dip>80:
            return rangeval[5]
    else:
        return hep
    
data['Schooling']=data[['Schooling','Income composition of resources']].apply(impute_ICR,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
corr_data[' thinness  1-19 years'].sort_values(ascending=False)

In [ ]:
corr_data[' thinness 5-9 years'].sort_values(ascending=False)

In [ ]:
sns.scatterplot(x=data['under-five deaths '],y=data[' thinness 5-9 years']);

In [ ]:
rangeval=[]
rangeval.append(data[data['under-five deaths ']<=100][' thinness 5-9 years'].mean())
rangeval.append(data[(100<data['under-five deaths '] ) & (data['under-five deaths ']<=250)][' thinness 5-9 years'].mean())
rangeval.append(data[(250<data['under-five deaths '] ) & (data['under-five deaths ']<=500)][' thinness 5-9 years'].mean())
rangeval.append(data[(500<data['under-five deaths '] ) & (data['under-five deaths ']<=800)][' thinness 5-9 years'].mean())
rangeval.append(data[(800<data['under-five deaths '] ) & (data['under-five deaths ']<=1000)][' thinness 5-9 years'].mean())
rangeval.append(data[(1000<data['under-five deaths '] ) & (data['under-five deaths ']<=1500)][' thinness 5-9 years'].mean())
rangeval.append(data[(1500<data['under-five deaths '] ) & (data['under-five deaths ']<=1750)][' thinness 5-9 years'].mean())
rangeval.append(data[data['under-five deaths ']>1750][' thinness 5-9 years'].mean())
print("sc<=25: "       ,rangeval[0])
print("100<sc<=250: "  ,rangeval[1])
print("250<sc<=500: "  ,rangeval[2])
print("500<sc<=800: "  ,rangeval[3])
print("800<sc<=1000: " ,rangeval[4])
print("1000<sc<=1500: ",rangeval[5])
print("1500<sc<=1750 " ,rangeval[6])
print("sc>1750: "      ,rangeval[7])
def impute_population(c):
    p=c[0]
    i=c[1]
    if pd.isnull(p):
        if i<=100:
            return  rangeval[0]
        elif 100<i<=250:
            return  rangeval[1]
        elif 250<i<=500:
            return  rangeval[2]
        elif 500<i<=800:
            return  rangeval[3]
        elif 800<i<=1000:
            return  rangeval[4]
        elif 1000<i<=1500:
            return  rangeval[5]
        elif 1500<i<=1750:
            return  rangeval[6]
        elif i>1750:
            return  rangeval[7]
    else:
        return p
    
data[' thinness 5-9 years']=data[[' thinness 5-9 years','under-five deaths ']].apply(impute_population,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
sns.scatterplot(x=data[' thinness 5-9 years'],y=data[' thinness  1-19 years']);

In [ ]:

rangeval=[]
rangeval.append(data[data[' thinness 5-9 years']<=5][' thinness  1-19 years'].mean())
rangeval.append(data[(5<data[' thinness 5-9 years'] ) & (data[' thinness 5-9 years']<=10)][' thinness  1-19 years'].mean())
rangeval.append(data[(10<data[' thinness 5-9 years'] ) & (data[' thinness 5-9 years']<=15)][' thinness  1-19 years'].mean())
rangeval.append(data[(15<data[' thinness 5-9 years'] ) & (data[' thinness 5-9 years']<=20)][' thinness  1-19 years'].mean())
rangeval.append(data[data[' thinness 5-9 years']>20][' thinness  1-19 years'].mean())
print("sc<=5: "       ,rangeval[0])
print("5<sc<=10: "  ,rangeval[1])
print("10<sc<=15: "  ,rangeval[2])
print("15<sc<=20: "  ,rangeval[3])
print("sc>20: "      ,rangeval[4])
def impute_population(c):
    p=c[0]
    i=c[1]
    if pd.isnull(p):
        if i<=100:
            return  rangeval[0]
        elif 100<i<=250:
            return  rangeval[1]
        elif 250<i<=500:
            return  rangeval[2]
        elif 500<i<=800:
            return  rangeval[3]
        elif 800<i<=1000:
            return  rangeval[4]
        elif 1000<i<=1500:
            return  rangeval[5]
        elif 1500<i<=1750:
            return  rangeval[6]
        elif i>1750:
            return  rangeval[7]
    else:
        return p
    
data[' thinness  1-19 years']=data[[' thinness  1-19 years',' thinness 5-9 years']].apply(impute_population,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(pd.isnull(data),cmap='hot')

In [ ]:
a=list(data.columns)
b=[]
for i in a:
    c=data[i].isnull().sum()
    b.append(c)
null_df=pd.DataFrame({'Feature name':a,'no. of Nan':b})
null_df

In [ ]:
data[' BMI '] = data[' BMI '].fillna(value=data[' BMI '].mean())
data['Income composition of resources'] = data['Income composition of resources'].fillna(value=data['Income composition of resources'].mean())
data['Schooling']= data['Schooling'].fillna(value=data['Schooling'].mean())

In [ ]:
sns.distplot(lifeExpectancy);

In [ ]:
data.info()

In [ ]:
data['Country'].unique()

In [ ]:
data['Country'].nunique()

In [ ]:
data['Status'].unique()

In [ ]:
Country_dummy = pd.get_dummies(data['Country'])

In [ ]:
Status_dummy = pd.get_dummies(data['Status'])

In [ ]:
data.drop(['Country','Status'],inplace=True,axis=1)

In [ ]:
data =pd.concat([data ,Country_dummy,Status_dummy],axis=1)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, lifeExpectancy, test_size=0.30, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression
Linear_model= LinearRegression()
Linear_model.fit(X_train,y_train)

In [ ]:
predictions1=Linear_model.predict(X_test)
predictions1[0:10]

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test,predictions1)**(0.5))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,predictions1)

In [ ]:
from sklearn.linear_model import Ridge
ridge_model=Ridge()
ridge_model.fit(X_train,y_train)

In [ ]:
predictions2=ridge_model.predict(X_test)
print(mean_squared_error(y_test,predictions2)**(0.5))

In [ ]:
from sklearn.linear_model import Lasso
lasso_model=Lasso(alpha=0.00000001)
lasso_model.fit(X_train,y_train)

In [ ]:
predictions3=lasso_model.predict(X_test)
print(mean_squared_error(y_test,predictions3)**(0.5))